In [1]:
import pickle
import numpy as np
import gc; gc.enable()
import os

In [2]:
SAVE_FOLDER = 'TF_IDF_W_BIN_C/'

In [3]:
with open('text_feature.pickle', 'rb') as handle:
    text_feature = pickle.load(handle)

In [4]:
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

In [6]:
tf_idf_features = TfidfVectorizer(ngram_range=(1, 2),
                                  analyzer='word',
                                  smooth_idf=False,
                                  sublinear_tf=True,
                                  max_features=50000,
                                  norm='l2', dtype=np.float32).fit_transform(text_feature.values)
print(tf_idf_features.shape)

(2011862, 50000)


In [7]:
with open(SAVE_FOLDER+'tf_idf_features.pickle', 'wb') as handle:
    pickle.dump(tf_idf_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
from scipy.sparse import hstack

tf_idf_features = hstack([tf_idf_features, (tf_idf_features > 0).astype(int)]).tocsr()
with open(SAVE_FOLDER+'tf_idf_features(with_bin).pickle', 'wb') as handle:
    pickle.dump(tf_idf_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
del tf_idf_features; gc.collect()

247

## Merge

In [10]:
import pickle
import pandas as pd
import numpy as np
import gc; gc.enable()
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack, save_npz

In [11]:
comm_feature = 'all_features.pickle' 
text_feature = SAVE_FOLDER+'tf_idf_features(with_bin).pickle'

In [12]:
with open(comm_feature, 'rb') as handle1:
    comm_feature = pickle.load(handle1).astype(np.float32)
    print(type(comm_feature))
    
with open(text_feature, 'rb') as handle2:
    text_feature = pickle.load(handle2)
    print(type(text_feature))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>


In [13]:
all_features = hstack([comm_feature, text_feature]).tocsr()

In [14]:
all_features = all_features.astype(np.float64)

In [15]:
all_features.shape

(2011862, 132042)

In [16]:
save_npz('all_features_tfidf_wordbin.npz', all_features)